# 다중 분류 실습(로이터 기사)
### 로이터(Reuters) 뉴스 데이터셋은 총 46개의 서로 다른 주제(클래스)로 분류된 뉴스 기사들로 구성되어 있다. 각 주제는 0부터 45까지의 정수로 인코딩되어 있다.

0: `cocoa`  
1: `grain`  
2: `veg-oil`  
3: `earn`  
4: `acq`  
5: `wheat`  
6: `copper`  
7: `housing`  
8: `money-supply`  
9: `coffee`  
10: `sugar`  
...  
46개의 주제는 레이블 값으로 train_label에 저장되어 있다.

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 로이터 기사 데이터셋 불러오기
from keras.datasets import reuters
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

2110848/2110848 [==============================] - 1s 1us/step


In [3]:
train_data.shape # 1차원 배열(벡터)이며 배열의 크기는 8982이다.

(8982,)

In [4]:
train_data[0] # 정수 인덱스로 매핑한 결과

[1,
 2,
 2,
 8,
 43,
 10,
 447,
 5,
 25,
 207,
 270,
 5,
 3095,
 111,
 16,
 369,
 186,
 90,
 67,
 7,
 89,
 5,
 19,
 102,
 6,
 19,
 124,
 15,
 90,
 67,
 84,
 22,
 482,
 26,
 7,
 48,
 4,
 49,
 8,
 864,
 39,
 209,
 154,
 6,
 151,
 6,
 83,
 11,
 15,
 22,
 155,
 11,
 15,
 7,
 48,
 9,
 4579,
 1005,
 504,
 6,
 258,
 6,
 272,
 11,
 15,
 22,
 134,
 44,
 11,
 15,
 16,
 8,
 197,
 1245,
 90,
 67,
 52,
 29,
 209,
 30,
 32,
 132,
 6,
 109,
 15,
 17,
 12]

###### 훈련 데이터의 첫 번째 기사를 디코딩하여 텍스트로 변환한다.

In [5]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()]) # 편의성을 위해 단어 사전과 반대로 인덱스를 key, 단어를 value로 지정한다.

decoded_newswire = ' '.join([reverse_word_index.get(i-3, '?') for i in train_data[0]]) # 원문으로 저장. 실제 단어가 3부터 시작. 찾지 못한 경우 '?' 반환

550378/550378 [==============================] - 1s 1us/step


In [6]:
decoded_newswire

'? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3'

In [7]:
train_labels[0]

3

In [8]:
# 데이터 준비
# 정수 인덱스로 매핑되어 있는 데이터를 원-핫 인코딩으로 변환하기 위한 함수
def to_one_hot_data(sequences, dimension=10000): # dimension 값은 단어 사전의 크기에 해당된다. 기본값: 10000
  results = np.zeros((len(sequences), dimension))

  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1

  return results

# 데이터 변환
x_train = to_one_hot_data(train_data)
x_test = to_one_hot_data(test_data)


In [9]:
train_labels

array([ 3,  4,  3, ..., 25,  3, 25])

In [15]:
# 레이블 준비
def to_one_hot_labels(labels, dimension=46): # 레이블 46개
   results = np.zeros((len(labels), dimension))
   # 만약 레이블이 1000개이고 dimension이 46이라면, 배열의 크기가 (1000, 46)인 2차원 배열이 생성되고 0으로 초기화된다.

   # sequence는 텍스트 데이터를 정수 인덱스로 변환한 시퀀스
   for i, sequence in enumerate(labels):
    results[i, sequence] = 1

   return results

one_hot_train_labels = to_one_hot_labels(train_labels)
one_hot_test_labels = to_one_hot_labels(test_labels)

###### results의 모든 값이 0으로 초기화된 상태이므로 [0, 0, 0, 0, ..., 0]과 같이 0이 46개인 배열의 형태이다. 레이블이 [1, 0, 0, 0, ..., 0], [0, 1, 0, 0, ..., 0]과 같이 표현된다. 첫 번째 데이터의 인덱스인 경우 i = 0이고 두 번째 데이터의 인덱스인 경우 i = 1이다.

###### 0: cocoa, 1: grain, 2: veg-oil, 3: earn, 4: acq, 5: wheat
[1, 0, 0, 0, ..., 0]
[0, 1, 0, 0, ..., 0]
[0, 0, 1, 0, ..., 0]

In [16]:
# 범주형 데이터로 변환하기
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

In [17]:
# 신경망 구성하기
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

###### 첫 번째 Dense Layer 추가
###### 두 번째 Dense Layer 추가. 앞선 출력이 자동으로 다음 레이어의 입력으로 전달된다.
###### 세 번째 Layer 추가. softmax 함수는 다중 클래스 분류 문제에서 각 클래스에 대한 확률을 출력하는 데 사용된다.

In [18]:
# 모델 요약 정보
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                640064    
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 46)                2990      
                                                                 
Total params: 647,214
Trainable params: 647,214
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer='rmsprop',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [20]:
# 데이터 분할 train:valid: = 9:1
val_dataset = x_train[:1000]
train_dataset = x_train[1000:]
val_label = one_hot_train_labels[:1000]
train_label = one_hot_train_labels[1000:]

In [65]:
history = model.fit(train_dataset, train_label, epochs=20, batch_size=512, validation_data=(val_dataset, val_label))
model.save('Sequential_model.h5')

Epoch 1/20
16/16 [==============================] - 1s 34ms/step - loss: 0.0693 - accuracy: 0.9627 - val_loss: 1.1230 - val_accuracy: 0.8050
Epoch 2/20
16/16 [==============================] - 0s 21ms/step - loss: 0.0694 - accuracy: 0.9593 - val_loss: 1.1274 - val_accuracy: 0.8070
Epoch 3/20
16/16 [==============================] - 0s 21ms/step - loss: 0.0686 - accuracy: 0.9588 - val_loss: 1.1433 - val_accuracy: 0.8050
Epoch 4/20
16/16 [==============================] - 0s 21ms/step - loss: 0.0706 - accuracy: 0.9599 - val_loss: 1.1454 - val_accuracy: 0.8030
Epoch 5/20
16/16 [==============================] - 0s 23ms/step - loss: 0.0684 - accuracy: 0.9580 - val_loss: 1.1505 - val_accuracy: 0.8130
Epoch 6/20
16/16 [==============================] - 0s 21ms/step - loss: 0.0707 - accuracy: 0.9600 - val_loss: 1.1592 - val_accuracy: 0.8060
Epoch 7/20
16/16 [==============================] - 0s 27ms/step - loss: 0.0688 - accuracy: 0.9587 - val_loss: 1.1280 - val_accuracy: 0.8070
Epoch 8/20
16

In [66]:
def evaluate_model(model, x_test, one_hot_test_labels):
    test_loss, test_acc = model.evaluate(x_test, one_hot_test_labels)
    return test_acc

# 모델 테스트 model.evaluate()
test_accuracy = evaluate_model(model, x_test, one_hot_test_labels)
print("Test Accuracy:", test_accuracy)

71/71 [==============================] - 0s 3ms/step - loss: 1.4012 - accuracy: 0.7769
Test Accuracy: 0.7769367694854736


In [67]:
from keras.models import load_model

# 저장된 모델의 파일 경로를 지정합니다.
model_path = 'Sequential_model.h5'

# 모델 불러오기
model = load_model(model_path)

In [68]:
# 모델 추론
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def tokenize_and_pad(text, word_index, max_length):
    tokenizer = Tokenizer(num_words=10000)
    tokenizer.word_index = word_index
    sequences = tokenizer.texts_to_sequences([text])
    padded_sequences = pad_sequences(sequences, maxlen=max_length)
    return padded_sequences

def preprocess_text(text, word_index, max_sequence_length=10000):
    # 정수 인덱스로 변환하는 함수
    words = text.lower().split()
    sequence = [word_index[word] if word in word_index else 0 for word in words] # 리스트 컴프리헨션
    sequence = sequence[:max_sequence_length]
    return sequence


def predict_category(model, text, word_index):
    # 새로운 기사를 입력받아 카테고리를 예측하는 함수
    sequence = preprocess_text(text, word_index)
    input_data = np.zeros((1, 10000))  # 모델이 기대하는 입력 형태로 변경
    for i, idx in enumerate(sequence):
        input_data[0, idx] = 1

    prediction = model.predict(input_data)
    predicted_category_index = np.argmax(prediction)
    return predicted_category_index

In [ ]:
"""
sequence = []

for word in words:
    if word in word_index:
        # word가 word_index 딕셔너리에 존재하는 경우, 인덱스를 sequence 리스트에 추가
        sequence.append(word_index[word])
    else:
        # word가 word_index 딕셔너리에 존재하지 않는 경우, 0을 sequence 리스트에 추가
        sequence.append(0)
"""

In [69]:
# 주제의 인덱스를 주제의 실제 레이블로 매핑하는 딕셔너리
category_index_to_label = {
    0: 'cocoa',
    1: 'grain',
    2: 'veg-oil',
    3: 'earn',
    4: 'acq',
    5: 'wheat',
    6: 'copper',
    7: 'housing',
    8: 'money-supply',
    9: 'coffee',
    10: 'sugar',
    11: 'trade',
    12: 'reserves',
    13: 'ship',
    14: 'cotton',
    15: 'carcass',
    16: 'crude',
    17: 'nat-gas',
    18: 'cpi',
    19: 'money-fx',
    20: 'interest',
    21: 'gnp',
    22: 'meal-feed',
    23: 'alum',
    24: 'oilseed',
    25: 'gold',
    26: 'tin',
    27: 'strategic-metal',
    28: 'livestock',
    29: 'retail',
    30: 'ipi',
    31: 'iron-steel',
    32: 'rubber',
    33: 'heat',
    34: 'jobs',
    35: 'lei',
    36: 'bop',
    37: 'zinc',
    38: 'orange',
    39: 'pet-chem',
    40: 'dlr',
    41: 'gas',
    42: 'silver',
    43: 'wpi',
    44: 'hog',
    45: 'lead'
}

In [70]:
article_path = 'article1.txt'

# 입력 텍스트를 토큰화하고 정수 인덱스로 변환
padded_input = tokenize_and_pad(article_path, word_index, max_length=10000)

# 모델을 사용하여 예측
predicted_category_index = predict_category(model, article_path, word_index)

# 인덱스를 실제 주제로 변환
predicted_category_label = category_index_to_label[predicted_category_index]
print(f'예측된 카테고리 인덱스: {predicted_category_index}, 주제: {predicted_category_label}')

1/1 [==============================] - 0s 49ms/step
예측된 카테고리 인덱스: 3, 주제: earn


In [71]:
article_path = 'article2.txt'

# 입력 텍스트를 토큰화하고 정수 인덱스로 변환
padded_input = tokenize_and_pad(article_path, word_index, max_length=10000)

# 모델을 사용하여 예측
predicted_category_index = predict_category(model, article_path, word_index)

# 인덱스를 실제 주제로 변환
predicted_category_label = category_index_to_label[predicted_category_index]
print(f'예측된 카테고리 인덱스: {predicted_category_index}, 주제: {predicted_category_label}')

1/1 [==============================] - 0s 19ms/step
예측된 카테고리 인덱스: 3, 주제: earn
